In [1]:
import pandas as pd

# Hypothesis: There is no significant difference between pageviews for health articles across categories of high, medium, low, and unknown levels of importance.

# Step 1: Load data and determine which variables you're using.

In [3]:
df = pd.read_csv('unique_health_articles.csv')

In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,article,qid,total_pageviews,description,category,page_size,revisions_since_2024
0,0,0,1977_NestlÃ©_boycott,Q1498774,27259,boycott,Low-importance,NaN,0
1,1,1,2007_tuberculosis_scare,Q4609852,550,international health scare,Low-importance,16565.0,13
2,2,2,2011_United_States_listeriosis_outbreak,Q4622783,871,disease outbreak in the United States,Low-importance,23708.0,13
3,3,3,2014_California_Proposition_46,Q119129906,153,NaN,Low-importance,5373.0,5
4,4,4,24_Hour_Fitness,Q4631849,1122,US fitness center chain,Low-importance,21019.0,53


Remove the unnamed columns.

In [6]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.tail()

,article,qid,total_pageviews,description,category,page_size,revisions_since_2024
878,Yoga_as_exercise,Q65091595,4324,physical activity consisting mainly of yoga poses,High-importance,105306.0,106
879,Young_Living,Q18067581,2947,American multi-level marketing company,Low-importance,23704.0,17
880,Zero_emission_zone,Q647266,127,defined area,High-importance,59.0,1
881,Zumba,Q229680,42089,dance fitness program,Mid-importance,22096.0,51
882,Zyzz,Q2618609,181047,Russian-born Australian bodybuilder and Intern...,Mid-importance,24173.0,41


The x variable is article of SOME level of importance (category), and the y variable is total_pageviews.

In [9]:
df.groupby('category')['total_pageviews'].mean()

category
High-importance       86956.530435
Low-importance        30896.722458
Mid-importance        63192.496933
Unknown-importance    58540.090226
Name: total_pageviews, dtype: float64